In [1]:
import os
import sys
import matplotlib.pyplot as plt

os.chdir('..')

import numpy as np
import torch
import pandas as pd

from models.TrainPlanningOperator3D import PlanningOperator3D

from planners import generaterandompos
from planners import AStarPlanner, FMMPlanner, PlanningOperatorPlanner



/home/sharath/anaconda3/envs/neuraloperator/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392036766/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Load Gibson Environment

In [2]:
Ntotal = 35
ntest = 5
sub = 1
Sx = Sy = Sz = int(((128 - 1) / sub) + 1)

maps = np.load('dataset/gibsonenv/occupancymap.npy', allow_pickle=True)
sdfs = np.load('dataset/gibsonenv/sdf.npy',allow_pickle= True)


maps = maps[:Ntotal][-ntest:, ::sub, ::sub, ::sub][:, :Sx, :Sy, :Sz]
sdfs = sdfs[:Ntotal][-ntest:, ::sub, ::sub, ::sub][:, :Sx, :Sy, :Sz]


Generate Random Positions

In [3]:
startpositions = generaterandompos(maps)
goalpositions = generaterandompos(maps)

Test on Single Maps

In [4]:
mapx = maps[0,:,:,:]
sdfx = sdfs[0,:,:,:]
startx = startpositions[0,:]
goalx = goalpositions[0,:]

# Model
modes = 12
width = 32
nlayers = 5

model = PlanningOperator3D(modes, modes, modes, width, nlayers)
model.load_state_dict(torch.load("trained_models/modeldijkstra256.ckpt"))
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


success,pathcost,planningtime = AStarPlanner(startx, goalx, mapx)

print("\nAStar\nSuccess:", success,
      "\nPathCost:", pathcost,
      "\nPlanningTime:",planningtime)


success,pathcost,planningtime = FMMPlanner(startx, goalx, mapx)
print("\nFMM\nSuccess:", success,
      "\nPathCost:", pathcost,
      "\nPlanningTime:",planningtime)



success,pathcost,planningtime = PlanningOperatorPlanner(startx, goalx, mapx, sdfx, model)
print("\nFMM\nSuccess:", success,
      "\nPathCost:", pathcost,
      "\nPlanningTime:",planningtime)



AStar
Success: True 
PathCost: 93.94306736958816 
PlanningTime: 0.9826841354370117

FMM
Success: True 
PathCost: 96.2820843738848 
PlanningTime: 0.43051862716674805
